<a href="https://colab.research.google.com/github/bvschwartz/selfie-ml/blob/master/Selfie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Download** and sort selfie data

In [0]:
!rm -rf master.zip selfie-ml-master wget.log
!wget https://github.com/bvschwartz/selfie-ml/archive/master.zip -o wget.log
!unzip -q master.zip
import sys
print(sys.path)
if not '/content/selfie-ml-master' in sys.path:
  print ("adding to path")
  sys.path.insert(0, '/content/selfie-ml-master')

In [0]:
from google.colab import output
def beep():
  output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

In [0]:
!wget "https://www.crcv.ucf.edu/data/Selfie/Selfie-dataset.tar.gz"
!tar xzf ./Selfie-dataset.tar.gz
!rm ./Selfie-dataset.tar.gz
beep()

In [0]:
import segment
segment.copy_all()

Training!

In [0]:
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import RMSprop

train_dir = 'dataset/training/color'
test_dir = 'dataset/testing/color'

datagen = ImageDataGenerator(rescale=1.0/255.0)
print("loading training data")
train_it = datagen.flow_from_directory(train_dir, class_mode='sparse', batch_size=32, target_size = (306, 306), color_mode="rgb")
cats = len(train_it.classes)
print(train_it.class_indices)
#print(train_it[0])
print("loading testing data")
test_it = datagen.flow_from_directory(test_dir, class_mode='sparse', batch_size=32, target_size = (306, 306), color_mode="rgb")
#print(test_it.class_indices)

model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(306, 306, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('cats') and 1 for the other ('dogs')
    tf.keras.layers.Dense(cats, activation='softmax')
    
    tf.keras.layers.Flatten(input_shape=(306, 306, 3)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(cats, activation='softmax')
])

model.summary()

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
history = model.fit_generator(
    train_it,
    steps_per_epoch=100,
    epochs=2,
    validation_steps=50,
    verbose=1,
    validation_data=test_it)


In [0]:
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop

train_dir = 'dataset/training/color'
test_dir = 'dataset/testing/color'

train_it = ImageDataGenerator(rescale=1.0/255)
print("loading training data")
train_it = train_it.flow_from_directory(train_dir, class_mode='sparse', batch_size=32, target_size = (306, 306), color_mode="rgb", shuffle=True)
cats = len(train_it.class_indices)
print("categories:", cats)
print("loading testing data")
test_it = ImageDataGenerator(rescale=1.0/255)
test_it = test_it.flow_from_directory(test_dir, class_mode='sparse', batch_size=32, target_size = (306, 306), color_mode="rgb", shuffle=False)

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(306, 306, 3)),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  #tf.keras.layers.Flatten(input_shape=(306, 306, 3)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(cats, activation='softmax')
])

model.summary()

In [0]:
(x, y) = train_it.next()
print(len(x), y)

In [0]:
# display some of the images and labels
train_it.reset()
(images, indexes) = train_it.next()
#print(indexes)
#print(np.histogram(indexes))
print(np.histogram([0, 0, 0, 1, 1, 2, 2, 2, 2], bins=3, range=[0, 3]))
print(np.sum([[1, 2, 3, 4], [4, 3, 2, 1], [10, 9, 8, 7]], axis=0))
xxx
# build the index -> category
index_to_label = [0] * len(train_it.class_indices)
#print(train_it.class_indices)
#print(train_it.num_classes)
for v, i in train_it.class_indices.items():
  #print("*", i, "*", v)
  index_to_label[i] = v
print(index_to_label)

# display the 32 images with category
import numpy as np
import matplotlib.pyplot as plt

print(images[0].shape)
for i in range(len(images)):
  image = images[i]
  label = index_to_label[int(indexes[i])]
  print(label)
  plt.figure()
  #print(x.shape)
  plt.imshow(image)
  #plt.colorbar()
  plt.grid(False)
  plt.show()


In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_it, epochs=5)
beep()

In [0]:
model.evaluate(test_it)

In [0]:
p = model.predict(test_it, verbose=1)
print(np.histogram(np.argmax(p, axis=1), bins=np.arange(0, cats + 1))[0])

In [0]:
#print(p[:5])
print(np.argmax(p[:40], axis=1))
print(np.histogram(np.argmax(p, axis=1), bins=[0, 1, 2, 3, 4]))
print(np.histogram(np.argmax(p, axis=1), bins=np.arange(0, cats + 1)))


In [0]:
n = test_it.next()
p = model.predict(n[0])
for i in range(len(p)):
  print("correct:", n[1][i], "argmax:", np.argmax(p[i]), "raw:", p[i])
#print(n[1])
#print(p)

In [0]:
import numpy as np
import matplotlib.pyplot as plt

n = train_it.next()
print(n[0].shape)
x = n[0][0]
print(n[0])
#print(train_it.classes)
#x = train_it.next()[0][0]
plt.figure()
print(x.shape)
plt.imshow(x)
plt.colorbar()
plt.grid(False)
plt.show()
